<img src="https://docs.dask.org/en/stable/_images/dask_horizontal.svg"
     align="right"
     width="40%"/>

# Dask Futures for simple parallelism

Dask futures are the foundation of all Dask APIs.  They are easy to use and flexible.  
Dask futures work with any Python function on any Python object.

## Create a few processes locally

In [ ]:
from dask.distributed import Client
client = Client()

client

## Some basic Python code

These functions pretend to do some work, but are very simple.  Dask doesn't care what code it runs.  You should imagine replacing these with your own code.

In [ ]:
import time, random

def inc(x):
    time.sleep(random.random())
    return x + 1

def dec(x):
    time.sleep(random.random())
    return x - 1

def add(x, y):
    time.sleep(random.random())
    return x + y

In [ ]:
%%time
inc(10)

## Sequential Code

This very simple code just calls these function ten times in a loop.

Dask makes it easy to parallelize simple code like this on your computer.

In [ ]:
%%time

results = []
for x in range(20):
    result = inc(x)
    result = dec(result)
    results.append(result)


## Parallel code


In [ ]:
%%time

results = []
for x in range(20):
    result = client.submit(inc, x)
    result = client.submit(dec, result)
    results.append(result)

results = client.gather(results)

## More complex code with tree reduction

The code above is very simple.  Let's show off that Dask can do more complex things.  
Here we add all of our elements pair-wise until there is only one left.  This looks especially fun if you bring up the "Graph" dashboard plot.

In [ ]:
%%time

results = []
for x in range(128):
    result = client.submit(inc, x)
    result = client.submit(dec, result)
    results.append(result)

# Add up all of the results, pairwise
while len(results) > 1:
    results = [
        client.submit(add, results[i],  results[i + 1]) 
        for i in range(0, len(results), 2)
    ]
    
results = client.gather(results)

## Scale Out

In [ ]:
import coiled

cluster = coiled.Cluster(
    n_workers=20,
)
client = cluster.get_client()

# Then rerun the cells above